# Extract Dataset

In [1]:
demo = False
data_dir = '../data/kaggle_dog'

if demo:
    zipfiles = ['train_valid_test_tiny.zip']
else:
    zipfiles = ['train.zip', 'test.zip', 'labels.csv.zip']

import zipfile
for fin in zipfiles:
    with zipfile.ZipFile(data_dir + '/' + fin, 'r') as zin:
        zin.extractall(data_dir)

FileNotFoundError: [Errno 2] No such file or directory: '../data/kaggle_dog/train.zip'

# Reorganize Dataset

In [2]:
import math
import os
import shutil
from collections import Counter

def reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir, 
                   valid_ratio):
    # read train_label
    with open(os.path.join(data_dir, label_file), 'r') as f:
        # jump 1st row of file(name of item)
        lines = f.readlines()[1:]
        tokens = [l.rstrip().split(',') for l in lines]
        idx_label = dict(((idx, label) for idx, label in tokens))
    labels = set(idx_label.values())
    
    num_train = len(os.listdir(os.path.join(data_dir, train_dir)))
    # num of the most less dog class
    min_num_train_per_label = (
        Counter(idx_label.values()).most_common()[:-2:-1][0][1])
    # num of every class of dog in valid_data
    num_valid_per_label = math.floor(min_num_train_per_label * valid_ratio)
    label_count = dict()
    
    def mkdir_if_not_exist(path):
        if not os.path.exists(os.path.join(*path)):
            os.makedirs(os.path.join(*path))
            
    # reorganize train_data and valid_data
    for train_file in os.listdir(os.path.join(data_dir, train_dir)):
        idx = train_file.split('.')[0]
        label = idx_label[idx]
        mkdir_if_not_exist([data_dir,input_dir, 'train_valid', label])
        shutil.copy(os.path.join(data_dir, train_dir, train_file), 
                    os.path.join(data_dir, input_dir, 'train_valid', label))
        if label not in label_count or label_count[label] < num_valid_per_label:
            mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file), 
                        os.path.join(data_dir, input_dir, 'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            mkdir_if_not_exist([data_dir, input_dir, 'train', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file), 
                        os.path.join(data_dir, input_dir, 'train', label))
            
        
    # reorganize test_data
    mkdir_if_not_exist([data_dir, input_dir, 'test', 'unknown'])
    for test_file in os.listdir(os.path.join(data_dir, test_dir)):
        shutil.copy(os.path.join(data_dir, test_dir, test_file), 
                    os.path.join(data_dir, input_dir, 'test', 'unknown'))

In [3]:
if demo:
    input_dir = 'train_valid_test_tiny'
    batch_size = 2
else:
    label_file = 'labels.csv'
    train_dir = 'train'
    test_dir = 'test'
    input_dir = 'train_valid_test'
    batch_size = 128
    valid_ratio = 0.1
    reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir, 
                   valid_ratio)

In [4]:
demo = False
data_dir = '../data/kaggle_dog'
label_file = 'labels.csv'
train_dir = 'train'
test_dir = 'test'
input_dir = 'train_valid_test'
batch_size = 128
valid_ratio = 0.1

# Read Dataset use Gluon

In [5]:
from mxnet import autograd as ag
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import ndarray as nd
from mxnet.gluon.data import vision
import numpy as np

def transform_train(data, label):
    im = image.imresize(data.astype('float32') / 255, 96, 96)
    auglist = image.CreateAugmenter(data_shape = (3, 96, 96), resize = 0, 
                                    rand_crop = False, rand_resize = False, rand_mirror = True, 
                                    mean = None, std = None, 
                                    brightness = 0, contrast = 0, 
                                    saturation = 0, hue = 0, 
                                    pca_noise = 0, rand_gray = 0, inter_method = 2)
    for aug in auglist:
        im = aug(im)
    # transpose data to format channels x height x width
    im = nd.transpose(im, (2, 0, 1))
    return (im, nd.array([label]).asscalar().astype('float32'))

def transform_test(data, label):
    im = image.imresize(data.astype('float32') / 255, 96, 96)
    im = nd.transpose(im, (2, 0, 1))
    return (im, nd.array([label]).asscalar().astype('float32'))

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [6]:
input_str = data_dir + '/' + input_dir + '/'

# read orginal image. flag = 1 means there is 3 channels in input image
train_ds = vision.ImageFolderDataset(input_str + 'train', flag = 1, 
                                     transform = transform_train)
valid_ds = vision.ImageFolderDataset(input_str + 'valid', flag = 1, 
                                     transform = transform_test)
train_valid_ds = vision.ImageFolderDataset(input_str + 'train_valid', 
                                           flag = 1, transform = transform_train)
test_ds = vision.ImageFolderDataset(input_str + 'test', flag = 1, 
                                    transform = transform_test)

loader = gluon.data.DataLoader
train_data = loader(train_ds, batch_size, shuffle = True, last_batch = 'keep')
valid_data = loader(valid_ds, batch_size, shuffle = True, last_batch = 'keep')
train_valid_data = loader(train_valid_ds, batch_size, shuffle = True, 
                          last_batch = 'keep')
test_data = loader(test_ds, batch_size, shuffle = False, last_batch = 'keep')

# Entropy Loss
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# Model

In [7]:
from mxnet.gluon import nn
from mxnet import ndarray as nd

class Residual(nn.HybridBlock):
    def __init__(self, channels, same_shape = True, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        with self.name_scope():
            strides = 1 if same_shape else 2
            self.conv1 = nn.Conv2D(channels, kernel_size = 3, padding = 1, 
                                   strides = strides)
            self.bn1 = nn.BatchNorm()
            self.conv2 = nn.Conv2D(channels, kernel_size = 3, padding = 1)
            self.bn2 = nn.BatchNorm()
            if not same_shape:
                self.conv3 = nn.Conv2D(channels, kernel_size = 1, 
                                       strides = strides)
    
    def hybrid_forward(self, F, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(out + x)
    
class ResNet(nn.HybridBlock):
    def __init__(self, num_classes, verbose = False, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            net = self.net = nn.HybridSequential()
            # modual 1
            net.add(nn.Conv2D(channels = 32, kernel_size = 3, strides = 1, 
                              padding = 1))
            net.add(nn.BatchNorm())
            net.add(nn.Activation(activation = 'relu'))
            # modual 2
            for _ in range(3):
                net.add(Residual(channels = 32))
            # modual 3
            net.add(Residual(channels = 64, same_shape = False))
            for _ in range(2):
                net.add(Residual(channels = 64))
            # modual 4
            net.add(Residual(channels = 128, same_shape = False))
            for _ in range(2):
                net.add(Residual(channels = 128))
            # modual 5
            net.add(nn.GlobalAvgPool2D())
            net.add(nn.Flatten())
            net.add(nn.Dense(num_classes))
        
    def hybrid_forward(self, F, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('Block %d output: %s' % (i + 1, out.shape))
        return out

def get_net(ctx):
    num_outputs = 120
    net = ResNet(num_outputs)
    net.initialize(ctx = ctx, init = init.Xavier())
    return net

# Train Model and Tune Params

In [8]:
import datetime
import sys
sys.path.append('..')
import utils

def get_loss(data, net, ctx):
    loss = 0.0
    for feas, label in data:
        label = label.as_in_context(ctx)
        output = net(feas.as_in_context(ctx))
        cross_entropy = softmax_cross_entropy(output, label)
        loss += nd.mean(cross_entropy).asscalar()
    return loss / len(data)

def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period, 
          lr_decay):
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9, 
                                      'wd': wd})
    
    prev_time = datetime.datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0.0
        if epoch > 0 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for data, label in train_data:
            label = label.as_in_context(ctx)
            with ag.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = 'Time %02d:%02d:%02d' % (h, m, s)
        if valid_data is not None:
            valid_loss = get_loss(valid_data, net, ctx)
            epoch_str = ('Epoch %d. Train loss: %f, Valid loss %f, ' 
                         % (epoch, train_loss / len(train_data), valid_loss))
        else:
            epoch_str = ('Epoch %d. Train loss: %f, ' 
                         % (epoch, train_loss / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str + ', lr' + str(trainer.learning_rate))

In [9]:
ctx = utils.try_gpu()
num_epochs = 100
learning_rate = 0.01
weight_decay = 5e-4
lr_period = 50
lr_decay = 0.1

net = get_net(ctx)
net.hybridize()
# print(net)
train(net, train_data, valid_data, num_epochs, learning_rate, 
      weight_decay, ctx, lr_period, lr_decay)

Epoch 0. Train loss: 4.834214, Valid loss 4.770660, Time 00:00:47, lr0.01
Epoch 1. Train loss: 4.637847, Valid loss 4.733740, Time 00:00:49, lr0.01
Epoch 2. Train loss: 4.511256, Valid loss 4.771920, Time 00:00:48, lr0.01
Epoch 3. Train loss: 4.388064, Valid loss 4.463984, Time 00:00:48, lr0.01
Epoch 4. Train loss: 4.276806, Valid loss 5.106804, Time 00:00:48, lr0.01
Epoch 5. Train loss: 4.165378, Valid loss 5.840738, Time 00:00:48, lr0.01
Epoch 6. Train loss: 4.068257, Valid loss 6.469334, Time 00:00:48, lr0.01
Epoch 7. Train loss: 3.964446, Valid loss 4.469808, Time 00:00:48, lr0.01
Epoch 8. Train loss: 3.855254, Valid loss 4.578573, Time 00:00:48, lr0.01
Epoch 9. Train loss: 3.778141, Valid loss 4.312709, Time 00:00:48, lr0.01
Epoch 10. Train loss: 3.700873, Valid loss 4.628507, Time 00:00:48, lr0.01
Epoch 11. Train loss: 3.603581, Valid loss 4.355377, Time 00:00:48, lr0.01
Epoch 12. Train loss: 3.523992, Valid loss 4.534934, Time 00:00:48, lr0.01
Epoch 13. Train loss: 3.487303, Val

# Predict Test_data

In [10]:
import numpy as np

net = get_net(ctx)
net.hybridize()
train(net, train_valid_data, None, num_epochs, learning_rate, weight_decay, 
      ctx, lr_period, lr_decay)

outputs = []
for data, label in test_data:
    output = nd.softmax(net(data.as_in_context(ctx)))
    outputs.extend(output.asnumpy())
ids = sorted(os.listdir(os.path.join(data_dir, input_dir, 'test/unknown')))
with open('submission.csv', 'w') as f:
    f.write('id,' + ','.join(train_valid_ds.synsets) + '\n')
    for i, output in zip(ids, outputs):
        f.write(i.split('.')[0] + ',' + ','.join(
            [str(num) for num in output]) + '\n')

Epoch 0. Train loss: 4.798259, Time 00:00:50, lr0.01
Epoch 1. Train loss: 4.610624, Time 00:00:49, lr0.01
Epoch 2. Train loss: 4.471569, Time 00:00:49, lr0.01
Epoch 3. Train loss: 4.361878, Time 00:00:49, lr0.01
Epoch 4. Train loss: 4.250830, Time 00:00:49, lr0.01
Epoch 5. Train loss: 4.159358, Time 00:00:49, lr0.01
Epoch 6. Train loss: 4.058501, Time 00:00:49, lr0.01
Epoch 7. Train loss: 3.971075, Time 00:00:49, lr0.01
Epoch 8. Train loss: 3.886227, Time 00:00:49, lr0.01
Epoch 9. Train loss: 3.805168, Time 00:00:49, lr0.01
Epoch 10. Train loss: 3.714724, Time 00:00:49, lr0.01
Epoch 11. Train loss: 3.609727, Time 00:00:49, lr0.01
Epoch 12. Train loss: 3.547778, Time 00:00:49, lr0.01
Epoch 13. Train loss: 3.470387, Time 00:00:49, lr0.01
Epoch 14. Train loss: 3.400170, Time 00:00:49, lr0.01
Epoch 15. Train loss: 3.322413, Time 00:00:49, lr0.01
Epoch 16. Train loss: 3.259262, Time 00:00:49, lr0.01
Epoch 17. Train loss: 3.185897, Time 00:00:49, lr0.01
Epoch 18. Train loss: 3.144804, Time 0